In [1]:
import spacy
import os
import pandas as pd

from src.preprocessing import convert_txt_to_json, convert_to_conllu

In [ ]:
# convert txt file to JSON file
input_file = "../data/chatgpt_dataset.txt"
output_file = "../data/chatgpt_dataset.json"
convert_txt_to_json(
    input_file=input_file, output_file=output_file, link="https://chatgpt.com/"
)

# convert JSON file to conllu dataset
output_file = os.path.join("../data", "chatgpt_dataset.conllu")
nlp = spacy.load("en_core_web_sm")
file_path = "../data/chatgpt_dataset.json"
data = pd.read_json(file_path, lines=True)
convert_to_conllu(data, output_file, nlp)

100%|██████████| 200/200 [00:01<00:00, 117.78it/s]
